<a href="https://colab.research.google.com/github/Kiana-M/Refreshers-and-Tutorials/blob/main/OOP_Mockinterview_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let’s dive into a mock interview question that ties in data quality monitoring and OOP fundamentals.

---

**Interview Question:**

"Let's design a basic **Data Quality Monitoring System** in Python. This system should allow us to track data quality metrics for different data sources, such as completeness and uniqueness. We'll structure it to follow OOP principles, making it modular and extensible for future metrics."

### Requirements

1. **Abstract Base Class**: Create an abstract base class `DataQualityMetric`, which defines the blueprint for all metrics.
   - This class should have an `evaluate` method that will be implemented in each concrete metric.

2. **Concrete Metrics**: Define at least two concrete classes that inherit from `DataQualityMetric`:
   - **CompletenessMetric**: Checks for missing values in a data source.
   - **UniquenessMetric**: Checks for duplicate entries in a data source.
   
3. **DataSource Class**: A `DataSource` class represents the data to be monitored. It has a method to load data and another method to run metrics on it.

4. **MetricEvaluator Class**: Finally, a class `MetricEvaluator` that takes in a `DataSource` and a list of `DataQualityMetric` objects, and evaluates each metric on the data source.

**Assume we’re working with a pandas DataFrame as the data source.**

---

### Example Usage

Here's a quick example of what this would look like when put together:

```python
import pandas as pd

# Define some data with missing values and duplicates for testing
data = pd.DataFrame({
    "id": [1, 2, 3, 4, None, 5, 2],
    "name": ["Alice", "Bob", None, "David", "Eve", "Frank", "Alice"]
})
```

Then, the code should work like this:

```python
source = DataSource(data)
evaluator = MetricEvaluator(source, [CompletenessMetric(), UniquenessMetric()])
results = evaluator.evaluate()
print(results)
```

### Let’s Begin the Design

1. **Step 1**: Start by defining the `DataQualityMetric` base class with the `evaluate` method.
  
2. **Step 2**: Implement `CompletenessMetric` and `UniquenessMetric` classes that inherit from `DataQualityMetric`.

---

Let’s start with **Step 1**: Go ahead and define the `DataQualityMetric` class as an abstract class, including the `evaluate` method.

Let me know if you'd like me to help clarify any part or proceed to the next steps!

In [ ]:

from abc import ABC, abstractmethod

class DataQualityMetric(ABC):

    @abstractmethod
    def evaluate(self):
        raise NotImplementedError("Subclasses must implement this method.")


class CompletenessMetric(DataQualityMetric):
    #Checks for missing values in a data source.

    def evaluate(self, df):
        if df.isnull().sum().sum() == 0:
            return False
        else:
            return True


class UniquenessMetric(DataQualityMetric):
    #Checks for duplicate entries in a data source.

    def evaluate(self, df):
        if len(df) == len(df.drop_duplicates()):
            return False
        else:
            return True

class DataSource:
    def __init__(self, data):
        self.__data = data

    @property
    def load_data(self):
        return self.__data


class MetricEvaluator:
    def __init__(self, source: DataSource, metric_list: list[DataQualityMetric]):
        self.data = source
        self.metric_list = metric_list

    def run_metrics(self):
        results = {}
        if self.data.load_data.empty:
            raise ValueError('No data')
        for metric in self.metric_list:
            metric_name = metric.__class__.__name__
            results[metric_name] = metric.evaluate(self.data.load_data)
        return results


In [ ]:
import pandas as pd

# Define some data with missing values and duplicates for testing
data = pd.DataFrame({
    "id": [1, 2, 3, 4, None, 5, 2],
    "name": ["Alice", "Bob", None, "David", "Eve", "Frank", "Alice"]
})


In [ ]:
source = DataSource(data)
evaluator = MetricEvaluator(source, [CompletenessMetric(), UniquenessMetric()])
results = evaluator.run_metrics()
print(results)

{'CompletenessMetric': True, 'UniquenessMetric': False}


# Interview Question

You’re tasked with designing a simple event monitoring system that can track and report various events within a data pipeline (e.g., data ingestion started, data validation failed, data export completed, etc.). The system needs to support different types of events and allow adding custom event types in the future. Your solution should be modular and flexible, allowing us to easily integrate it into a larger system.

Requirements:

	1.	Event Base Class: Create an Event base class that will represent a general event. Each event should have a name (str) and a timestamp (use datetime.now()).
	2.	Event Types: Define three specific event classes that inherit from Event:
	  •	DataIngestionEvent - triggered when data ingestion starts. It should include source (str) as an additional attribute.
	  •	DataValidationEvent - triggered when data validation fails. It should include validation_errors (list of str).
	  •	DataExportEvent - triggered when data export completes. It should include destination (str).
	3.	Event Logger: Create an EventLogger class responsible for tracking and reporting events. It should have:
	  •	A method log_event that accepts an Event instance and stores it.
	  •	A method get_report that returns a summary of all logged events in a human-readable format.
	  •	Optionally, add a way to filter events based on their type (e.g., get all DataValidationEvent logs).
	4.	Future Compatibility: Design the code so adding a new event type, like DataTransformationEvent, would require minimal changes to the existing system.

Bonus:

Make Event a bit more interesting by enforcing that each subclass must implement a get_details method that provides information specific to the event type (e.g., data source or validation errors). This method should be called by EventLogger when generating the report.

Take a few minutes to think about the structure and let me know when you’re ready to start. If you get stuck, I’ll give you some pointers.


In [2]:
from abc import ABC, abstractmethod
from datetime import datetime

class Event(ABC):
    def __init__(self, name: str):
        self.name = name
        self.time_stamp = datetime.now()

    @abstractmethod
    def get_report(self):
        raise NotImplementedError("Defined in concrete classes")

class DataIngestionEvent(Event):
    # triggered when data ingestion starts
    def __init__(self, name: str, source: str):
        super().__init__(name)
        self._source = source  # Using single underscore for encapsulation

    @property
    def source(self):
        return self._source

    def get_report(self):
        return f"{self.time_stamp} - Data ingestion started for {self.name} from source {self.source}"

class DataValidationEvent(Event):
    # triggered when data validation fails
    def __init__(self, name: str, validation_errors: list[str]):
        super().__init__(name)
        self.__validation_errors = validation_errors

    @property
    def validation_errors(self):
        return self.__validation_errors

    def get_report(self):
        message = f"{self.time_stamp} - Data validation failed for {self.name}. Error List:"
        for error in self.validation_errors:
            message += '\n' + error
        return message

class DataExportEvent(Event):
    # triggered when data export completes
    def __init__(self, name: str, destination: str):
        super().__init__(name)
        self.__destination = destination

    @property
    def destination(self):
        return self.__destination

    def get_report(self):
        return f"{self.time_stamp} - Data export completed for {self.name} to destination {self.destination}"

class EventLogger:
    def __init__(self):
        self.events = []  # Initialize as an empty list

    def log_event(self, event: Event):
        self.events.append(event)

    def get_reports(self):
        for event in self.events:
            print(event.get_report())
